In [1]:
! pip install python-dotenv
! pip install langchain --upgrade


[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python3 -m pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 13.7 MB/s eta 0:00:00 0:00:01
  Attempting uninstall: langchain
    Found existing installation: langchain 0.0.245
    Uninstalling langchain-0.0.245:
      Successfully uninstalled langchain-0.0.245

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
import os
import sys

path = "/var/genie-in-the-box/src/lib"
sys.path.append( path )
print( sys.path )

import util as du
du.add_to_path( path )

from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.agents.agent_toolkits.pandas.base import create_pandas_dataframe_agent

import pandas as pd


from IPython.display import display, Markdown


['/Applications/PyCharm.app/Contents/plugins/python/helpers-pro/jupyter_debug', '/Applications/PyCharm.app/Contents/plugins/python/helpers/pydev', '/var', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages', '/var/genie-in-the-box/src/lib']
Path [/var/genie-in-the-box/src/lib] already in sys.path


In [3]:
# llm    = ChatOpenAI( model_name="gpt-3.5-turbo", temperature=0.0 )
#
# memory = ConversationBufferMemory( memory_key="chat_history", return_messages=True )

# qa = ConversationalRetrievalChain.from_llm(
#     llm,
#     in_memory_jsonl_index.vectorstore.as_retriever(),
#     memory=memory
# )

# def query_embeddings( query ):
#
#     result = qa( { "question": query } )
#
#     return result[ "answer" ]
#
# result = query_embeddings( "How Many times have I asked you about birthdays?" )
# result

In [4]:
# llm = ChatOpenAI( model_name="gpt-4", temperature=0.0 )

llm    = ChatOpenAI( model_name="gpt-3.5-turbo", temperature=0.0 )
memory = ConversationBufferMemory( memory_key="chat_history", return_messages=True )
raw_df = pd.read_csv( du.get_project_root() + "/src/conf/long-term-memory/events.csv" )

df_agent        = create_pandas_dataframe_agent( llm, raw_df, verbose=True )
df_agent.memory = memory

In [5]:
memory

ConversationBufferMemory(chat_memory=ChatMessageHistory(messages=[]), output_key=None, input_key=None, return_messages=True, human_prefix='Human', ai_prefix='AI', memory_key='chat_history')

In [6]:
df_agent.agent.template

AttributeError: 'ZeroShotAgent' object has no attribute 'template'

In [ ]:
df_agent.run( "What day is today? Use tool python_repl_ast to respond." )



> Entering new AgentExecutor chain...


Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: HTTPSConnectionPool(host='api.openai.com', port=443): Max retries exceeded with url: /v1/chat/completions (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f3c7aeb47f0>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution')).
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: HTTPSConnectionPool(host='api.openai.com', port=443): Max retries exceeded with url: /v1/chat/completions (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f3c7ace8d00>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution')).
Retrying langchain.chat_models.openai.ChatOpenAI.com

In [169]:
df_agent.run( "How many records are in this dataset?" )



> Entering new AgentExecutor chain...
Thought: I can use the `len()` function to find the number of records in the dataset.
Action: python_repl_ast
Action Input: len(df)
Observation: 200
Thought:There are 200 records in this dataset.
Final Answer: 200

> Finished chain.


'200'

In [173]:
df_agent.run( "Today is 2023-07-27. What is tomorrow's date? Use tool python_repl_ast" )



> Entering new AgentExecutor chain...
Thought: I can use the `datetime` module in Python to calculate tomorrow's date.
Action: python_repl_ast
Action Input: `from datetime import datetime, timedelta`
Observation: 
Thought:I have imported the necessary module. Now I can use the `datetime` module to calculate tomorrow's date.
Action: python_repl_ast
Action Input: `today = datetime(2023, 7, 27)`
Observation: 
Thought:I have created a `datetime` object for today's date. Now I can use the `timedelta` function to calculate tomorrow's date.
Action: python_repl_ast
Action Input: `tomorrow = today + timedelta(days=1)`
Observation: 
Thought:I have calculated tomorrow's date successfully.
Final Answer: Tomorrow's date is 2023-07-28.

> Finished chain.


"Tomorrow's date is 2023-07-28."

In [174]:
df_agent.run( "Today is 2023-07-27. What date was yesterday? Use tool python_repl_ast" )



> Entering new AgentExecutor chain...
Thought: To find the date of yesterday, I can subtract one day from the current date.

Action: python_repl_ast
Action Input: from datetime import date, timedelta
Observation: 
Thought:I have imported the necessary modules to work with dates and time intervals. Now I can proceed to subtract one day from the current date.

Action: python_repl_ast
Action Input: current_date = date(2023, 7, 27)
Observation: 
Thought:I have defined the current date as July 27, 2023. Now I can subtract one day from this date to find yesterday's date.

Action: python_repl_ast
Action Input: yesterday = current_date - timedelta(days=1)
Observation: 
Thought:I have subtracted one day from the current date and stored the result in the variable "yesterday". Now I can print the value of "yesterday" to find out the date of yesterday.

Action: python_repl_ast
Action Input: print(yesterday)
Observation: 2023-07-26

Thought:Yesterday's date was July 26, 2023.
Final Answer: July 2

'July 26, 2023'

In [180]:
df_agent.run( "Today is 2023-07-27. What events do i have today? Use tool python_repl_ast" )



> Entering new AgentExecutor chain...
Thought: I need to filter the dataframe to only include events that have the same start_date as today.

Action: python_repl_ast
Action Input: df[df['start_date'] == '2023-07-27']
Observation:     start_date    end_date  duration start_time end_time    event_type  \
71  2023-07-27  2023-07-27         0      00:00    23:59   Anniversary   
72  2023-07-27  2023-07-30         3      00:00    23:59  Subscription   
73  2023-07-27  2023-07-28         1      05:03    06:50          TODO   
74  2023-07-27  2023-07-27         0      22:07    03:01          TODO   

    recurrent recurrence_interval                      description  \
71       True              2 week  Gregorio (friend)'s anniversary   
72       True               3 day          This is a subscription.   
73      False                 NaN                  This is a todo.   
74      False                 NaN                  This is a todo.   

   priority_level  
71           high  
72    

'The events I have today are:\n- Anniversary (recurrent event)\n- Subscription (recurrent event)\n- TODO (non-recurrent event)\n- TODO (non-recurrent event)'

In [189]:
df_agent.run( "Today is 2023-07-27. What events do i have today? Use tool python_repl_ast" )



> Entering new AgentExecutor chain...
Thought: I need to filter the dataframe to only include events that have the same start_date as today.

Action: python_repl_ast
Action Input: df[df['start_date'] == '2023-07-27']
Observation:     start_date    end_date  duration start_time end_time    event_type  \
71  2023-07-27  2023-07-27         0      00:00    23:59   Anniversary   
72  2023-07-27  2023-07-30         3      00:00    23:59  Subscription   
73  2023-07-27  2023-07-28         1      05:03    06:50          TODO   
74  2023-07-27  2023-07-27         0      22:07    03:01          TODO   

    recurrent recurrence_interval                      description  \
71       True              2 week  Gregorio (friend)'s anniversary   
72       True               3 day          This is a subscription.   
73      False                 NaN                  This is a todo.   
74      False                 NaN                  This is a todo.   

   priority_level  
71           high  
72    

'The events I have today are:\n- Anniversary (recurrent event)\n- Subscription (recurrent event)\n- TODO (non-recurrent event)\n- TODO (non-recurrent event)'

In [179]:
raw_df[ raw_df[ 'start_date' ] == '2023-07-27' ][ [ 'event_type', 'start_time' ] ]

,event_type,start_time
73,TODO,05:03
74,TODO,22:07
71,Anniversary,00:00
72,Subscription,00:00


In [181]:
response = df_agent.run( "What events do i have today, when do they start? Use tool python_repl_ast" )



> Entering new AgentExecutor chain...
Thought: I need to filter the dataframe to only include events that have the start_date equal to today's date. Then I can select the event_type and start_time columns to see what events I have today and when they start.

Action: python_repl_ast
Action Input: df[df['start_date'] == pd.to_datetime('today').date()][['event_type', 'start_time']]
Observation: Empty DataFrame
Columns: [event_type, start_time]
Index: []
Thought:

OutputParserException: Could not parse LLM output: `There are no events today.`

In [182]:
memory

ConversationBufferMemory(chat_memory=ChatMessageHistory(messages=[HumanMessage(content='What day is today? Use tool python_repl_ast to respond.', additional_kwargs={}, example=False), AIMessage(content='2023-07-27', additional_kwargs={}, example=False), HumanMessage(content='How many records are in this dataset?', additional_kwargs={}, example=False), AIMessage(content='200', additional_kwargs={}, example=False), HumanMessage(content='What day is tomorrow? Use tool python_repl_ast', additional_kwargs={}, example=False), AIMessage(content="Tomorrow's date is stored in the variable `tomorrow`.", additional_kwargs={}, example=False), HumanMessage(content='What date is tomorrow? Use tool python_repl_ast', additional_kwargs={}, example=False), AIMessage(content="Tomorrow's date is 2023-07-03.", additional_kwargs={}, example=False), HumanMessage(content="Today is 2023-07-27. What is tomorrow's date? Use tool python_repl_ast", additional_kwargs={}, example=False), AIMessage(content="Tomorrow'

In [183]:
df_agent.run( "What have we discussed thus far? Use tool python_repl_ast" )



> Entering new AgentExecutor chain...


OutputParserException: Could not parse LLM output: `Thought: We have discussed the structure and contents of the dataframe `df`.

Action: None
`

In [184]:
df_agent.run( "Today is July 27th, 2023. Do I have any events today? Use tool python_repl_ast" )



> Entering new AgentExecutor chain...
Thought: To check if there are any events today, I need to compare the current date with the start_date and end_date columns in the dataframe.

Action: Use python_repl_ast
Action Input: `df[(df['start_date'] <= '2023-07-27') & (df['end_date'] >= '2023-07-27')]`
Observation: Use python_repl_ast is not a valid tool, try another one.
Thought:I can use the `query` method in pandas to filter the dataframe based on the conditions.

Action: Use python_repl_ast
Action Input: `df.query("'2023-07-27' >= start_date and '2023-07-27' <= end_date")`
Observation: Use python_repl_ast is not a valid tool, try another one.
Thought:I can use the `loc` method in pandas to filter the dataframe based on the conditions.

Action: Use python_repl_ast
Action Input: `df.loc[(df['start_date'] <= '2023-07-27') & (df['end_date'] >= '2023-07-27')]`
Observation: Use python_repl_ast is not a valid tool, try another one.
Thought:I can use the `loc` method in pandas to filter the 

KeyboardInterrupt: 

In [185]:
df_agent.run( "Today is July 27th, 2023. What kind of event do I have today? Use tool python_repl_ast" )



> Entering new AgentExecutor chain...
Thought: To find out what kind of event I have today, I need to compare the current date with the start_date and end_date columns in the dataframe.

Action: Use the pandas `datetime` module to get the current date and compare it with the start_date and end_date columns.

Action Input: 
```python_repl_ast
import pandas as pd
from datetime import datetime

current_date = datetime(2023, 7, 27).date()
df['start_date'] = pd.to_datetime(df['start_date']).dt.date
df['end_date'] = pd.to_datetime(df['end_date']).dt.date
event_today = df[(df['start_date'] <= current_date) & (df['end_date'] >= current_date)]
event_today['event_type']
```

Observation: Use the pandas `datetime` module to get the current date and compare it with the start_date and end_date columns. is not a valid tool, try another one.
Thought:

OutputParserException: Could not parse LLM output: `I need to use the `python_repl_ast` tool to execute the code and get the result.`

In [186]:
response = df_agent.run( "What kind of event do I have for July 27th, 2023? Tell me their description and what time they start. Use markdown to format the output. Use tool python_repl_ast" )



> Entering new AgentExecutor chain...
Thought: To find the event for July 27th, 2023, I need to filter the dataframe based on the start_date column.

Action: Use the pandas loc function to filter the dataframe.

Action Input: `df.loc[df['start_date'] == '2023-07-27']`

Observation: Use the pandas loc function to filter the dataframe. is not a valid tool, try another one.
Thought:The pandas loc function is not a valid tool in this case. I should try using the pandas query function instead.

Action: Use the pandas query function to filter the dataframe.

Action Input: `df.query("start_date == '2023-07-27'")`

Observation: Use the pandas query function to filter the dataframe. is not a valid tool, try another one.
Thought:The pandas query function is also not a valid tool in this case. I should try using boolean indexing instead.

Action: Use boolean indexing to filter the dataframe.

Action Input: `df[df['start_date'] == '2023-07-27']`

Observation: Use boolean indexing to filter the d

KeyboardInterrupt: 

In [91]:
Markdown( response )

|    | event_type   | description             | start_time   |
|---:|:-------------|:------------------------|:-------------|
|  0 | Performance  | This is a performance.  | 00:00        |
|  1 | Interview    | This is a interview.    | 15:31        |

In [84]:
response = df_agent.run( "What kind of event do I have for July 27th, 2023? Tell me their description and what time they start. Use tool python_repl_ast" )



> Entering new AgentExecutor chain...
Thought: I need to filter the dataframe to only include events on July 27th, 2023 and then select the columns for event type, description, and start time.

Action: Use the `loc` function to filter the dataframe and select the desired columns.

Action Input: `df.loc[df['date'] == '2023-07-27', ['event_type', 'description', 'start_time']]`

Observation: Use the `loc` function to filter the dataframe and select the desired columns. is not a valid tool, try another one.
Thought:I need to use the `python_repl_ast` tool to execute the command.

Action: Use the `python_repl_ast` tool.

Action Input: `python_repl_ast`

Observation: Use the `python_repl_ast` tool. is not a valid tool, try another one.
Thought:I need to use the `python_repl_ast` tool to execute the command.

Action: Use the `python_repl_ast` tool.

Action Input: `python_repl_ast`

Observation: Use the `python_repl_ast` tool. is not a valid tool, try another one.
Thought:I need to use the `

In [187]:
response = df_agent.run( "Show me the events I have on July 28th, 2023. Use tool python_repl_ast" )
response



> Entering new AgentExecutor chain...
Thought: I need to filter the dataframe to only include events on July 28th, 2023.

Action: Use the `loc` function to filter the dataframe.

Action Input: `df.loc[df['start_date'] == '2023-07-28']`

Observation: Use the `loc` function to filter the dataframe. is not a valid tool, try another one.
Thought:

OutputParserException: Could not parse LLM output: `I need to use the `python_repl_ast` tool to execute the command.`

In [188]:
response = df_agent.run( "What events do I have for July 28th, 2023. Use tool python_repl_ast" )
response



> Entering new AgentExecutor chain...
Thought: I need to filter the dataframe to only include events that occur on July 28th, 2023.

Action: Use the `loc` function to filter the dataframe based on the `start_date` column.

Action Input: `df.loc[df['start_date'] == '2023-07-28']`

Observation: Use the `loc` function to filter the dataframe based on the `start_date` column. is not a valid tool, try another one.
Thought:I need to use the `query` function instead of the `loc` function to filter the dataframe based on the `start_date` column.

Action: Use the `query` function to filter the dataframe based on the `start_date` column.

Action Input: `df.query("start_date == '2023-07-28'")`

Observation: Use the `query` function to filter the dataframe based on the `start_date` column. is not a valid tool, try another one.
Thought:I can use the `loc` function to filter the dataframe based on the `start_date` column.

Action: Use the `loc` function to filter the dataframe based on the `start_

'Agent stopped due to iteration limit or time limit.'